In [1]:
import pygsti
from scipy.linalg import expm
import numpy as np
from pygsti.tools import unitary_to_superop
from pygsti.modelpacks import smq1Q_XYZI
from matplotlib import pyplot as plt
from pygsti.circuits import Circuit
from quapack.pyRPE import RobustPhaseEstimation
from quapack.pyRPE.quantum import Q as _rpeQ
from tqdm import tqdm
from importlib import reload

In [36]:
def PauliMatrix(idx):
    if idx == 0:
        return np.eye(2)
    elif idx == 1:
        return np.array([[0, 1], [1, 0]])
    elif idx == 2:
        return np.array([[0, -1j], [1j, 0]])
    elif idx == 3:
        return np.array([[1, 0], [0, -1]])
    else:
        raise ValueError('Invalid index for Pauli matrix')
    
def PauliTensor(idx1, idx2):
    return np.kron(PauliMatrix(idx1), PauliMatrix(idx2))

def make_generator(pauli):
    dimension = pauli.shape[0]
    I = np.eye(dimension)
    return -(1j/2)*(np.kron(I, pauli) - np.kron(pauli.T, I))

def decimal_to_quaternary(n):
    if n == 0:
        return '0'
    quaternary = ''
    while n:
        n, remainder = divmod(n, 4)
        quaternary = str(remainder) + quaternary
    return quaternary

def decimal_to_pauli_string(idx, num_qubits):
    quaternary = decimal_to_quaternary(idx).zfill(num_qubits)
    return ''.join([['I', 'X', 'Y', 'Z'][int(digit)] for digit in quaternary])

def decimal_to_pauli_tensor(idx, num_qubits):
    quaternary = decimal_to_quaternary(idx).zfill(num_qubits)
    tensor = 1
    for digit in quaternary:
        tensor = np.kron(tensor, PauliMatrix(int(digit)))
    return tensor

In [37]:
zz_generator = make_generator(PauliTensor(3, 3))
zz_pmat = expm((np.pi/2)*zz_generator)

In [38]:
def make_sorted_eigsystem(pmat):
    eigvals, eigvecs = np.linalg.eig(pmat)
    idxs = np.argsort(np.angle(eigvals))
    return eigvals[idxs], eigvecs[:, idxs]

def make_perturbation_vector(pmat, generator):
    eigvals, eigenvectors = make_sorted_eigsystem(pmat)
    perturbations = []
    for e in eigenvectors:
        p = np.conj(e).T @ generator @ e
        perturbations.append(p)
    return np.array(perturbations)

In [39]:
def make_pertubation_dict(pmat, generators, interleaving_operators):
    pertubation_dict = {}
    for k, v in interleaving_operators.items():
        pertubation_dict[k] = {}
        for kg, g in generators.items():
            pertubation_dict[k][kg] = make_perturbation_vector(pmat@v, g)
    return pertubation_dict
        

In [60]:
import itertools

In [79]:
# make the set of all single qubit clifford rotations
clifford_generators = {
    'ri': np.eye(4),
    'rx': expm((np.pi/2)*make_generator(PauliMatrix(1))), 
    'ry': expm((np.pi/2)*make_generator(PauliMatrix(2))),
    'rz': expm((np.pi/2)*make_generator(PauliMatrix(3)))
}
num_products = 4
all_product_strings = [i for i in itertools.product(clifford_generators.keys(), repeat=num_products)]
complete_dict = {}
for product in all_product_strings:
    complete_dict[product] = np.linalg.multi_dot([clifford_generators[gen] for gen in product])
# now remove duplicates from the dictionary
unique_dict = {}
for k, v in complete_dict.items():
    if not any([np.allclose(v, u) for u in unique_dict.values()]):
        unique_dict[k] = v
all_clifford_operations_1q = unique_dict

In [80]:
print(len(unique_dict), 'unique products')
print(len(complete_dict), 'total products')

24 unique products
256 total products


In [84]:
# make the tensor product of all single qubit clifford rotations
all_separable_clifford_operations = {}
for k, v in all_clifford_operations_1q.items():
    for k2, v2 in all_clifford_operations_1q.items():
        all_separable_clifford_operations[(k, k2)] = np.kron(v, v2)

In [85]:
len(all_separable_clifford_operations)

576

In [86]:
# make all the hamiltonian generators
all_hamiltonian_generators = {}
for i in range(16):
    pstring = decimal_to_pauli_string(i, 2)
    generator = make_generator(decimal_to_pauli_tensor(i, 2))
    all_hamiltonian_generators[pstring] = generator

In [87]:
pdict = make_pertubation_dict(zz_pmat, all_hamiltonian_generators, all_separable_clifford_operations)

In [93]:
def find_max_perturbation(pdict, generator_string):
    max_perturb = 0
    max_interleaving_op = -1
    for interleaving_op_string in pdict.keys():
        perturbation = pdict[interleaving_op_string][generator_string]
        perturb = max(abs(perturbation))
        if perturb > max_perturb:
            max_perturb = perturb
            max_interleaving_op = interleaving_op_string
    return max_perturb, max_interleaving_op

def rank_top_perturbations(pdict, generator_string, num_top_perturbations=5):
    perturbations = []
    for interleaving_op_string in pdict.keys():
        perturbation = pdict[interleaving_op_string][generator_string]
        perturbations.append((interleaving_op_string, max(abs(perturbation))))
    perturbations.sort(key=lambda x: x[1], reverse=True)
    return perturbations[:num_top_perturbations]

def list_all_perterbations(pdict, generator_string):
    perturbations = []
    for interleaving_op_string in pdict.keys():
        perturbation = pdict[interleaving_op_string][generator_string]
        perturbations.append((interleaving_op_string, max(abs(perturbation))))
    return perturbations

In [103]:
rank_top_perturbations(pdict, 'YY')

[((('ri', 'ri', 'rx', 'rz'), ('ri', 'ri', 'ry', 'rx')), 1.0168843355176842),
 ((('ri', 'ri', 'rx', 'rz'), ('rx', 'rx', 'ry', 'rx')), 1.0168843355176842),
 ((('ri', 'ri', 'rz', 'ry'), ('rx', 'rx', 'rx', 'ry')), 0.7403765980501658),
 ((('ri', 'ri', 'rz', 'ry'), ('rx', 'ry', 'ry', 'ry')), 0.7403765980501653),
 ((('ri', 'rx', 'rx', 'ry'), ('ri', 'rx', 'rx', 'rx')), 0.7265026283895843)]